In [1]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 1) DataFrame을 생성

In [2]:
import os
from pyspark.sql.types import StructType, StructField, StringType
police=spark.read\
    .options(header="true", delimiter=" ", inferSchema="true")\
    .schema(
        StructType([
            StructField("sent",StringType()),
            ])
    )\
    .text(os.path.join("data", "20191021_policeAddress.txt"))

In [3]:
police.show(9, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sent                                                                                                                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.                                                                                                                                                                           |
|                                                                     

### 2) 단어로 분리해서, 출력

In [4]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="tokens")
tokDf = tokenizer.transform(police)

In [5]:
tokDf.show(9, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sent                                                                                                                                                                                                                                        |tokens                                                                                                                                                                                                                                                            

In [6]:
for r in tokDf.select("sent").take(9):
    print (r[0])

존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.

국민의 안전을 위해 밤낮없이 애쓰시는 전국의 15만 경찰관 여러분께 먼저 감사를 드립니다. 전몰·순직 경찰관들의 고귀한 희생에 경의를 표합니다. 유가족 여러분께 위로의 마음을 전합니다.

오늘 홍조근정훈장을 받으신 중앙경찰학교장 이은정 치안감님, 근정포장을 받으신 광주남부경찰서 김동현 경감님을 비롯한 수상자 여러분께 각별한 축하와 감사를 드립니다. 또한 경찰 영웅으로 추서되신 차일혁, 최중락님께 국민의 사랑을 전해드립니다.

사랑하는 경찰관 여러분,

여러분의 헌신적 노력으로 우리의 치안은 참 좋아졌습니다. 지난해 범죄 발생은 2015년에 비해 15.1% 줄었습니다. 같은 기간 교통사고 사망자는 18.2% 감소했습니다.


###  3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [7]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="tokens", outputCol="nostops")
stop.setStopWords([u"돌", u"참", u"더", u"그", u"등", u"법", u"큰"])

StopWordsRemover_a6a2ecca8cb4

In [8]:
for e in stop.getStopWords():
    print (e, end="/")

돌/참/더/그/등/법/큰/

### 4) 불용어 제거하고, 출력

In [9]:
stopDf = stop.transform(tokDf)
stopDf.show(9, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sent                                                                                                                                                                                                                                  

In [10]:
for r in stopDf.select("nostops").take(9):
    for e in r:
        print (e)

['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '‘경찰의', '날’입니다.']
['']
['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.']
['']
['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.']
['']
['사랑하는', '경찰관', '여러분,']
['']
['여러분의', '헌신적', '노력으로', '우리의', '치안은', '좋아졌습니다.', '지난해', '범죄', '발생은', '2015년에', '비해', '15.1%', '줄었습니다.', '같은', '기간', '교통사고', '사망자는', '18.2%', '감소했습니다.']


### 5) TF-IDF를 계산하고, 출력 & 6) TF-IDF 컬럼을 features로 구성, 출력

In [11]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)

In [12]:
idf = IDF(inputCol="hash", outputCol="idf")

idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [13]:
idfDf.select("nostops", "hash").show(9, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nostops                                                                                                                                                                                                                                                                 |hash                                                                                                                                                                                 